In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

from qubic.lib.Qgps import GPSCalsource, GPSAntenna, GPStools
 
%matplotlib inline

In [ ]:
data_path = "calsource_orientation_20240322.dat"

In [ ]:
distance_between_antennas = 1
distance_calsource = 0.5
gps_antenna = GPSAntenna(data_path, distance_between_antennas)

In [ ]:
gps_antenna.plot_gps_data()

In [ ]:
position_ini_antenna1 = gps_antenna.position_antenna1[:, 9615]
position_ini_antenna2 = gps_antenna.position_antenna2[:, 9615]
# position_ini_calsource = ((position_ini_antenna2 + position_ini_antenna1)/2)
position_ini_calsource = np.array([1, 2, 1])
observation_date = np.array([dt.datetime(2024, 3, 22, 10, 8, 00)]) # np.array([dt.datetime(2024, 3, 22, 9, 47, 49), dt.datetime(2024, 3, 22, 16, 13, 49, 500000)])#
gps = GPSCalsource(data_path, position_ini_antenna1, position_ini_antenna2, position_ini_calsource, distance_between_antennas, observation_date, position_qubic = np.array([0, 0, 0]), ini_wrt_antenna2=False)

In [ ]:
%matplotlib widget
gps.plot_system()